In [58]:
# tester.ipynb
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))


# from src.analyzer import analyze_speech
from src.analyzer_raw import analyze_speech

audio_path = "../samples/sample8-ielts8.5.wav"
# audio_path = "../samples/sample7.flac"
# audio_path = "../data/l2arctic_spontaneous/L2A_001.wav"

result = analyze_speech(audio_path)
result

Analyzing audio: ../samples/sample8-ielts8.5.wav
Context: conversational

[1/5] Transcribing with Whisper (verbatim)...
  Duration: 101.82s
  Words: 186

[2/5] Marking filler words...
  Marked: 7 filler words
  Content words: 179

[3/5] Aligning words with WhisperX...
  Aligned: 186 words

[4/5] Detecting subtle fillers with Wav2Vec2...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Total events: 22

[5/5] Calculating raw score...


{'raw_transcript': 'Sure. Well, uh, one time what actually helps someone need was, uh, through a charity group I volunteered with. Uh, the name of the group was helping hands. Uh, the person I assisted was a homeless man, uh, and his name was John actually, who was struggling to find out stable housing and employment. Uh, the situation aroused during a, uh, community outreach and event organized by the charity group. When we provided food, clothing and resources those who need, John approached our booth looking quite lost in this heartbeat. He actually shared his story about how he formed hard times after losing his job and how difficult it was for him to get back on his feet. I took the initiative to sit down with him and listen to his concerns. I also, at the same time, I helped him fill out fords for local shelters and job replacement services. After that, I provided him with the care package that included food, hygiene products and information available in the area. After that I fe

In [59]:
from src.llm_processing import extract_llm_annotations, aggregate_llm_metrics
from src.prosody_extraction import is_monotone_speech

llm_result = extract_llm_annotations(result["raw_transcript"])
llm_metrics = aggregate_llm_metrics(llm_result)
is_monotone = is_monotone_speech(audio_path)

llm_metrics, is_monotone

Prosody Variation (F0 std): 12.13 Hz
Monotone Detected: True


({'coherence_breaks': 0,
  'topic_relevance': True,
  'word_choice_errors': 4,
  'advanced_vocabulary_count': 0,
  'complex_structures_attempted': 3,
  'complex_structures_accurate': 0,
  'grammar_errors': 4,
  'meaning_blocking_error_ratio': 0.0},
 True)

In [60]:
from datetime import datetime

def build_master_result(
    result: dict,
    llm_metrics: dict,
    is_monotone: bool,
) -> dict:
    # ---- Lexical density ----
    total_words = result["statistics"]["total_words_transcribed"]
    content_words = result["statistics"]["content_words"]

    # ---- Word confidence metrics ----
    confidences = [
        w["confidence"]
        for w in result["timestamps"]["words_timestamps_raw"]
        if w.get("confidence") is not None
    ]

    mean_word_confidence = (
        sum(confidences) / len(confidences) if confidences else 0.0
    )

    low_confidence_ratio = (
        sum(1 for c in confidences if c < 0.7) / len(confidences)
        if confidences else 0.0
    )

    return {
        "metadata": {
            "audio_duration_sec": round(result["audio_duration_sec"], 2),
            "speaking_time_sec": round(result["speaking_time_sec"], 2),
            "total_words_transcribed": total_words,
            "content_word_count": content_words,
            "analysis_timestamp": datetime.utcnow().isoformat() + "Z",
        },

        "fluency_coherence": {
            "pauses": {
                "pause_frequency_per_min": result.get("pause_frequency"),
                "long_pause_rate": result["long_pauses_per_min"],
                "pause_variability": float(result["pause_variability"]),
            },
            "rate": {
                "speech_rate_wpm": float(result["wpm"]),
                "speech_rate_variability": float(result["speech_rate_variability"]),
            },
            "disfluency": {
                "filler_frequency_per_min": float(result["fillers_per_min"]),
                "stutter_frequency_per_min": float(result["stutters_per_min"]),
                "repetition_rate": float(result["repetition_ratio"]),
            },
            "coherence": {
                "coherence_breaks": llm_metrics["coherence_breaks"],
                "topic_relevance": llm_metrics["topic_relevance"],
            },
        },

        "lexical_resource": {
            "breadth": {
                "unique_word_count": int(
                    round(result["vocab_richness"] * content_words)
                ),
                "lexical_diversity": float(result["vocab_richness"]),
                "lexical_density": round(result["lexical_density"], 3),
                "most_frequent_word_ratio": float(result["repetition_ratio"]),
            },
            "quality": {
                "word_choice_errors": llm_metrics["word_choice_errors"],
                "advanced_vocabulary_count": llm_metrics["advanced_vocabulary_count"],
            },
        },

        "grammatical_range_accuracy": {
            "complexity": {
                "mean_utterance_length": float(result["mean_utterance_length"]),
                "complex_structures_attempted": llm_metrics["complex_structures_attempted"],
                "complex_structures_accurate": llm_metrics["complex_structures_accurate"],
            },
            "accuracy": {
                "grammar_errors": llm_metrics["grammar_errors"],
                "meaning_blocking_error_ratio": llm_metrics["meaning_blocking_error_ratio"],
            },
        },

        "pronunciation": {
            "intelligibility": {
                "mean_word_confidence": round(mean_word_confidence, 3),
                "low_confidence_ratio": round(low_confidence_ratio, 3),
            },
            "prosody": {
                "monotone_detected": is_monotone,
            },
        },

        "raw_data": {
            "word_timestamps": result["timestamps"]["words_timestamps_raw"],
            "pause_events": result.get("pause_events", []),
            "filler_events": result["timestamps"]["filler_timestamps"],
            "stutter_events": result.get("stutter_events", []),
        },
    }


master_result = build_master_result(
    result, llm_metrics, is_monotone= is_monotone,
)


master_result


{'metadata': {'audio_duration_sec': 101.82,
  'speaking_time_sec': np.float64(91.72),
  'total_words_transcribed': 186,
  'content_word_count': 179,
  'analysis_timestamp': '2026-01-14T18:02:26.947421Z'},
 'fluency_coherence': {'pauses': {'pause_frequency_per_min': 7.071302298173247,
   'long_pause_rate': 1.7678255745433118,
   'pause_variability': 0.3317675345323854},
  'rate': {'speech_rate_wpm': 105.48025928108427,
   'speech_rate_variability': 0.2572295225970058},
  'disfluency': {'filler_frequency_per_min': 4.242781378903949,
   'stutter_frequency_per_min': 0.0,
   'repetition_rate': 0.06306306306306306},
  'coherence': {'coherence_breaks': 0, 'topic_relevance': True}},
 'lexical_resource': {'breadth': {'unique_word_count': 122,
   'lexical_diversity': 0.6815642458100558,
   'lexical_density': 0.597,
   'most_frequent_word_ratio': 0.06306306306306306},
  'quality': {'word_choice_errors': 4, 'advanced_vocabulary_count': 0}},
 'grammatical_range_accuracy': {'complexity': {'mean_utte

In [61]:
import importlib
import sys

# Force reload of the module
if 'scripts.ielts_band_scorer' in sys.modules:
    del sys.modules['scripts.ielts_band_scorer']

from scripts.ielts_band_scorer import IELTSBandScorer

scorer = IELTSBandScorer()
band_scores = scorer.score(master_result)

band_scores

{'overall_band': 5.0,
 'criterion_bands': {'fluency_coherence': 6.0,
  'lexical_resource': 6.5,
  'grammatical_range_accuracy': 4.5,
  'pronunciation': 6.0},
 'feedback_summary': 'Weakest: Grammar (4.5), Strongest: Lexis (6.5)',
 'duration_factor_applied': {'threshold_mult': 1.0, 'penalty_mult': 1.0}}

In [62]:
print("Grammar metrics:")
print(f"  Attempts: {master_result['grammatical_range_accuracy']['complexity']['complex_structures_attempted']}")
print(f"  Accurate: {master_result['grammatical_range_accuracy']['complexity']['complex_structures_accurate']}")
print(f"  Errors: {master_result['grammatical_range_accuracy']['accuracy']['grammar_errors']}")
print(f"\nLexical metrics:")
print(f"  Advanced vocab count: {master_result['lexical_resource']['quality']['advanced_vocabulary_count']}")
print(f"  Word choice errors: {master_result['lexical_resource']['quality']['word_choice_errors']}")


Grammar metrics:
  Attempts: 3
  Accurate: 0
  Errors: 4

Lexical metrics:
  Advanced vocab count: 0
  Word choice errors: 4


In [63]:
print("=" * 70)
print("BAND SCORING EXPLANATION")
print("=" * 70)
print(f"\n✓ Overall Band: {band_scores['overall_band']}")
print(f"\nCriteria Breakdown:")
for criterion, score in band_scores['criterion_bands'].items():
    label = criterion.replace('_', ' ').title()
    print(f"  • {label}: {score}")

print(f"\n{band_scores['feedback_summary']}")

print("\n" + "=" * 70)
print("KEY IMPROVEMENTS IN V2 SCORER:")
print("=" * 70)
print("""
✓ Grammar: Stricter accuracy thresholds (72% for Band 6, not 60%)
✓ Fluency: Penalizes fillers at 3.5/min (was 6.0), coherence breaks at 0+ (was 1+)
✓ Lexical: Capped at 6.5 without advanced vocabulary
✓ Pronunciation: Stricter confidence thresholds (0.80 for Band 6, not 0.82)
✓ Overall: Weakness gap enforcement prevents inflated overall from single strength
""")


BAND SCORING EXPLANATION

✓ Overall Band: 5.0

Criteria Breakdown:
  • Fluency Coherence: 6.0
  • Lexical Resource: 6.5
  • Grammatical Range Accuracy: 4.5
  • Pronunciation: 6.0

Weakest: Grammar (4.5), Strongest: Lexis (6.5)

KEY IMPROVEMENTS IN V2 SCORER:

✓ Grammar: Stricter accuracy thresholds (72% for Band 6, not 60%)
✓ Fluency: Penalizes fillers at 3.5/min (was 6.0), coherence breaks at 0+ (was 1+)
✓ Lexical: Capped at 6.5 without advanced vocabulary
✓ Pronunciation: Stricter confidence thresholds (0.80 for Band 6, not 0.82)
✓ Overall: Weakness gap enforcement prevents inflated overall from single strength

